In [2]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [3]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Lamar Jackson,BAL,QB,4,115.3
1,Josh Allen,BUF,QB,4,111.4
2,Nick Chubb,CLE,RB,4,85.6
3,Jalen Hurts,PHI,QB,4,101.3
4,Saquon Barkley,NYG,RB,4,86.0
...,...,...,...,...,...
276,Jonnu Smith,NWE,TE,4,12.8
289,George Kittle,SFO,TE,2,11.2
290,Isaiah Likely,BAL,TE,4,10.1
293,Albert Okwuegbunam,DEN,TE,4,10.5


### 2. Get Projections

##### 2a. My Projections

In [7]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [8]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}
games_played = {'ARI' : 4,
 'ATL' : 4,
 'BAL' : 4,
 'BUF' : 4,
 'CAR' : 4,
 'CHI' : 4,
 'CIN' : 4,
 'CLE' : 4,
 'DAL' : 4,
 'DEN' : 4,
 'DET' : 4,
 'GNB' : 4,
 'HOU' : 4,
 'IND' : 4,
 'JAX' : 4,
 'KAN' : 4,
 'LAC' : 4,
 'LAR' : 4,
 'LVR' : 4,
 'MIA' : 4,
 'MIN' : 4,
 'NWE' : 4,
 'NOR' : 4,
 'NYG' : 4,
 'NYJ' : 4,
 'PHI' : 4,
 'PIT' : 4,
 'SEA' : 4,
 'SFO' : 4,
 'TAM' : 4,
 'TEN' : 4,
 'WAS' : 4}


In [11]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

In [16]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = newerPts['Tm'].replace(games_played)
    newerPts['PPR_adj'] = newerPts['PPR'] * newerPts['GP']
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
# newerPts.to_excel('test.xlsx')

In [17]:
newerPts['Points'].isnull().sum()

0

In [18]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.45860180850322446
0.3698634789879465
0.48391461542643577


0      461.2
1      445.6
2      342.4
3      405.2
4      344.0
       ...  
262     51.2
263     44.8
264     40.4
265     42.0
266     40.4
Name: PPR_adj, Length: 267, dtype: float64